## 准备数据

In [25]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [26]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [27]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(shape=[28*28,100],dtype=tf.float32,
                              initial_value=tf.random.uniform(shape=[28*28, 100],minval=-0.5, maxval=0.5))
        self.W2 = tf.Variable(shape=[100,10],dtype=tf.float32,initial_value=tf.random.uniform(shape=[100, 10],minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[100],initial_value=tf.zeros(100))
        self.b2 = tf.Variable(shape=[10],initial_value=tf.zeros(10))
        self.trainable_variables = [self.W1, self.W2, self.b1, self.b2]
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        print("=======================logits=========================",logits)
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

# def __init__(self):
        
#         self.W1 = np.random.normal(size=[28*28+1, 100])
#         self.W2 = np.random.normal(size=[100, 10])
        
#         self.mul_h1 = Matmul()
#         self.mul_h2 = Matmul()
#         self.relu = Relu()
#         self.softmax = Softmax()
#         self.log = Log()

## 计算 loss

In [28]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [29]:
train_data, test_data = mnist_dataset()
for epoch in range(100):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

=======================logits========================= Tensor("add_1:0", shape=(60000, 10), dtype=float32)
epoch 0 : loss 2.3326824 ; accuracy 0.11748333
epoch 1 : loss 2.3227649 ; accuracy 0.123566665
epoch 2 : loss 2.3129725 ; accuracy 0.13043334
epoch 3 : loss 2.3033016 ; accuracy 0.13681667
epoch 4 : loss 2.2937484 ; accuracy 0.14351666
epoch 5 : loss 2.284309 ; accuracy 0.15068333
epoch 6 : loss 2.2749803 ; accuracy 0.158
epoch 7 : loss 2.2657585 ; accuracy 0.16488333
epoch 8 : loss 2.2566411 ; accuracy 0.17066666
epoch 9 : loss 2.2476246 ; accuracy 0.17825
epoch 10 : loss 2.2387068 ; accuracy 0.18545
epoch 11 : loss 2.2298844 ; accuracy 0.1922
epoch 12 : loss 2.2211552 ; accuracy 0.19863333
epoch 13 : loss 2.212517 ; accuracy 0.20585
epoch 14 : loss 2.2039666 ; accuracy 0.2124
epoch 15 : loss 2.1955025 ; accuracy 0.2186
epoch 16 : loss 2.1871226 ; accuracy 0.22498333
epoch 17 : loss 2.178825 ; accuracy 0.23186667
epoch 18 : loss 2.170607 ; accuracy 0.23815
epoch 19 : loss 2.16246